# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [430]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json

# Import API key
from api_keys import myKey

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [431]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
countries = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1750)  
lngs = np.random.uniform(low=-180.000, high=180.000, size=1750)
lat_lngs = zip(lats, lngs)
#uniform is to have the numbers evenly distributed
#size is how many random numbers will be generated.
#we are saying, generate 1500 random evenly distributed numbers between -90 and 90.

# Identify nearest city for each lat, lng combination
for each in lat_lngs:
    city = citipy.nearest_city(each[0], each[1]).city_name 
    country = citipy.nearest_city(each[0], each[1]).country_code 
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        countries.append(country)

# Print the city count to confirm sufficient count
len(cities)

#612 cites are returned.   There must be at least 500 on the next part of finding the weather.  If not, then must come back to this step and increase size.

685

In [432]:
len(countries)

685

In [433]:
#??  how do I see the ln_lngs list?
print(lat_lngs)

In [434]:
print(lngs)

[ -12.34842006    8.1594146  -161.3364408  ... -160.61345879   23.56613824
  118.96701327]


In [435]:
#

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [436]:
#first, perform a check on a single city.

In [437]:
url = "http://api.openweathermap.org/data/2.5/weather?"
city = "alofi"
units = "imperial"

# Build query URL
query_url = url + "appid=" + myKey + "&q=" + city + "&units=imperial"
#note:   adding imperial units will return the temperature in farenheight and the wind speed in miles per hour
print(query_url)

http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&q=alofi&units=imperial


In [438]:
# Get weather data
weather_response = requests.get(query_url)
weather_json = weather_response.json()


In [439]:
print(json.dumps(weather_json, indent=4, sort_keys=True))

{
    "base": "stations",
    "clouds": {
        "all": 68
    },
    "cod": 200,
    "coord": {
        "lat": -19.06,
        "lon": -169.92
    },
    "dt": 1554058800,
    "id": 4036284,
    "main": {
        "humidity": 94,
        "pressure": 1010,
        "temp": 78.8,
        "temp_max": 78.8,
        "temp_min": 78.8
    },
    "name": "Alofi",
    "sys": {
        "country": "NU",
        "id": 7306,
        "message": 0.0043,
        "sunrise": 1554053160,
        "sunset": 1554096080,
        "type": 1
    },
    "weather": [
        {
            "description": "broken clouds",
            "icon": "04d",
            "id": 803,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 260,
        "speed": 9.17
    }
}


In [440]:
#Success!   Able to retrieve a single city.
#now retrieve each individual piece of data needed for the loop

In [441]:
print(weather_json["name"])

Alofi


In [442]:
print(weather_json["coord"]["lat"])

-19.06


In [443]:
print(weather_json["main"]["temp"])

78.8


In [444]:
print(weather_json["main"]["humidity"])

94


In [445]:
print(weather_json["clouds"]["all"])

68


In [446]:
print(weather_json["wind"]["speed"])

9.17


### need a dataframe containing city and country to begin
NOTE:   the Weather API will take only a city; however, if multiples are found, API responds with a list of results that match a searching word. 
<BR>
Therefore, I will begin with a list of cities and countries which can also be passed to the 
    <BR>q city name and country code divided by comma, use ISO 3166 country codes
        
<BR> https://openweathermap.org/current

In [447]:
#Begin with the city and country list which was created in the begining.

cityCountry = pd.DataFrame(list(zip(cities, countries)),
              columns=['city','country'])

cityCountry.head()

,city,country
0,atar,mr
1,naters,ch
2,bethel,us
3,husavik,is
4,cabo san lucas,mx


In [448]:
#confirm there are at least 500 countries
len(cityCountry)

685

In [449]:
#  ??? not sure if I will take this approach

#the weather API allows for no more than 60 calls per minutes.
#divide the list into groups of 60
#https://openweathermap.org/price

# Add column for batch
# Note that we used "" to specify initial entry.
cityCountry["batch"] = ""
cityCountry.head()

,city,country,batch
0,atar,mr,
1,naters,ch,
2,bethel,us,
3,husavik,is,
4,cabo san lucas,mx,


In [450]:
# create a data frome with the values needed for the API calls

cityCountry2 = cityCountry
cityCountry2["location"] = (cityCountry2["city"] + ", " + cityCountry2["country"])
cityCountry2["newCity"] = ""
cityCountry2["newCountry"] = ""
cityCountry2["lat"] = ""
cityCountry2["long"] = ""
cityCountry2["tempF"] = ""
cityCountry2["humidity"] = ""
cityCountry2["cloud"] = ""
cityCountry2["windMPH"] = ""
cityCountry2["maxTemp"] = ""
cityCountry2.head()

,city,country,batch,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp
0,atar,mr,,"atar, mr",,,,,,,,,
1,naters,ch,,"naters, ch",,,,,,,,,
2,bethel,us,,"bethel, us",,,,,,,,,
3,husavik,is,,"husavik, is",,,,,,,,,
4,cabo san lucas,mx,,"cabo san lucas, mx",,,,,,,,,


### Populate the data only for a few records
Practice before attemping to get data for 500+ cities

In [451]:
#create a new df with only 10 records
cityCountrySample = cityCountry2.head(10)
cityCountrySample.head(20)

,city,country,batch,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp
0,atar,mr,,"atar, mr",,,,,,,,,
1,naters,ch,,"naters, ch",,,,,,,,,
2,bethel,us,,"bethel, us",,,,,,,,,
3,husavik,is,,"husavik, is",,,,,,,,,
4,cabo san lucas,mx,,"cabo san lucas, mx",,,,,,,,,
5,vaini,to,,"vaini, to",,,,,,,,,
6,butaritari,ki,,"butaritari, ki",,,,,,,,,
7,plouzane,fr,,"plouzane, fr",,,,,,,,,
8,tuatapere,nz,,"tuatapere, nz",,,,,,,,,
9,ushuaia,ar,,"ushuaia, ar",,,,,,,,,


In [452]:
#ensure I can get a single response using parameters
parameters = {"appid": myKey,
             "units": "imperial",
             "q": "vaini, to"}
base_url = "http://api.openweathermap.org/data/2.5/weather?"

In [453]:
#make the response in json format by adding .json()
singleResponse = requests.get(base_url, params=parameters)

In [454]:
print(singleResponse.url)

http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=vaini%2C+to


In [455]:
singleResponseJ = singleResponse.json()

In [456]:
print(json.dumps(singleResponseJ, indent=4, sort_keys=True))

{
    "base": "stations",
    "clouds": {
        "all": 20
    },
    "cod": 200,
    "coord": {
        "lat": -21.2,
        "lon": -175.2
    },
    "dt": 1554058800,
    "id": 4032243,
    "main": {
        "humidity": 94,
        "pressure": 1010,
        "temp": 77,
        "temp_max": 77,
        "temp_min": 77
    },
    "name": "Vaini",
    "sys": {
        "country": "TO",
        "id": 7285,
        "message": 0.0041,
        "sunrise": 1554140885,
        "sunset": 1554183652,
        "type": 1
    },
    "visibility": 10000,
    "weather": [
        {
            "description": "few clouds",
            "icon": "02d",
            "id": 801,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 140,
        "speed": 8.05
    }
}


In [457]:
print(singleResponseJ["name"])

Vaini


In [458]:
#success!  I can get a valid URL using parameters.  

In [459]:
#begin by knowing the number of cities
print("Number of cities to begin: " + str(len(cityCountry))) 

Number of cities to begin: 685


In [469]:
pd.options.mode.chained_assignment = None  # default='warn'

In [470]:
# create a parameters dict that will be updated with new city each iteration
parameters = {"appid": myKey,
             "units": "imperial"}

print("---------------------------------------------") 
print("Beginning Data Retrieval")
print("---------------------------------------------") 
      
#begin by knowing the number of cities
print("Number of cities to obtain data: " + str(len(cityCountry))) 
print("---------------------------------------------")       

# Loop through the cityCountrySample and perform a search on each
for index, row in cityCountrySample.iterrows():
    base_url = "http://api.openweathermap.org/data/2.5/weather?"

    city = row['city']
    country = row['country']

    # update address key value
    parameters['q'] = f"{city},{country}"

    # make request
    cities_data = requests.get(base_url, params=parameters)
    
    #print the record number  ??? how do I print the index number?
    print("Retrieving index # " + str(index) + " | " + row["city"] + ", " + row["country"])
      
    #print the city which is being retrieved
    #print(cityCountrySample)
    
    #confirm the URL
    print(cities_data.url)
        
    # convert to json
    citiesJ = cities_data.json()
    
    try:
        cityCountrySample.loc[index, "newCity"] = citiesJ["name"]
    except:
        print("     Missing city name... skipping data.")     
    
    try:    
        cityCountrySample.loc[index, "newCountry"] = citiesJ["sys"]["country"]
    except: 
        print("     Missing country name... skipping data.")
 
    try:    
        cityCountrySample.loc[index, "lat"] = citiesJ["coord"]["lat"]
    except: 
        print("     Missing latitude... skipping data.")
        
    try:    
        cityCountrySample.loc[index, "long"] = citiesJ["coord"]["lon"]
    except: 
        print("     Missing longitude... skipping data.")
        
    try:    
        cityCountrySample.loc[index, "tempF"] = citiesJ["main"]["temp"]
    except: 
        print("     Missing temp(F)... skipping data.")
        
    try:    
        cityCountrySample.loc[index, "humidity"] = citiesJ["main"]["humidity"]
    except: 
        print("     Missing humidity... skipping data.")
        
    try:    
        cityCountrySample.loc[index, "cloud"] = citiesJ["clouds"]["all"]
    except: 
        print("     Missing %of cloudiness... skipping data.")
     
    try:    
        cityCountrySample.loc[index, "windMPH"] = citiesJ["wind"]["speed"]
    except: 
        print("     Missing wind speed... skipping data.")
        
    try:    
        cityCountrySample.loc[index, "maxTemp"] = citiesJ["main"]["temp_max"]
    except: 
        print("     Missing maximum temperature... skipping data.")
    
    #the openweather API is free for up to 60 calls per minute.  Add a sleep timer to wait .75 seconds between calls.
    time.sleep(.75)

print("---------------------------------------------")   
print("Date Retrieval Complete")
print("---------------------------------------------")   
    
# Visualize to confirm new city and new country appear
cityCountrySample.head()


---------------------------------------------
Beginning Data Retrieval
---------------------------------------------
Number of cities to obtain data: 685
---------------------------------------------
Retrieving index # 0 | atar, mr
http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=atar%2Cmr
Retrieving index # 1 | naters, ch
http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=naters%2Cch
Retrieving index # 2 | bethel, us
http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=bethel%2Cus
Retrieving index # 3 | husavik, is
http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=husavik%2Cis
Retrieving index # 4 | cabo san lucas, mx
http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=cabo+san+lucas%2Cmx
Retrieving index # 5 | vaini, to
http://a

,city,country,batch,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp
0,atar,mr,,"atar, mr",Atar,MR,20.52,-13.05,59.06,58,0,4.47,59.06
1,naters,ch,,"naters, ch",Naters,CH,46.32,7.99,54.55,53,0,9.17,59
2,bethel,us,,"bethel, us",Bethel,US,60.79,-161.76,37.4,85,1,3.47,37.4
3,husavik,is,,"husavik, is",Husavik,IS,66.04,-17.34,41,65,75,17.22,41
4,cabo san lucas,mx,,"cabo san lucas, mx",Cabo San Lucas,MX,22.89,-109.91,81.37,53,5,11.41,84.99


In [466]:
cityCountrySample.head(10)

,city,country,batch,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp
0,atar,mr,,"atar, mr",Atar,MR,20.52,-13.05,59.06,58,0,4.47,59.06
1,naters,ch,,"naters, ch",Naters,CH,46.32,7.99,54.55,53,0,9.17,59
2,bethel,us,,"bethel, us",Bethel,US,60.79,-161.76,37.4,85,1,3.47,37.4
3,husavik,is,,"husavik, is",Husavik,IS,66.04,-17.34,41,65,75,17.22,41
4,cabo san lucas,mx,,"cabo san lucas, mx",Cabo San Lucas,MX,22.89,-109.91,81.73,53,5,11.41,86
5,vaini,to,,"vaini, to",Vaini,TO,-21.2,-175.2,77,94,20,8.05,77
6,butaritari,ki,,"butaritari, ki",Butaritari,KI,3.07,172.79,82.55,100,68,15.43,82.55
7,plouzane,fr,,"plouzane, fr",Plouzane,FR,48.38,-4.62,53.33,71,20,9.17,55.99
8,tuatapere,nz,,"tuatapere, nz",Tuatapere,NZ,-46.13,167.69,55.99,80,24,10,55.99
9,ushuaia,ar,,"ushuaia, ar",Ushuaia,AR,-54.81,-68.31,52,81,75,19.46,53.6


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot